In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [ ]:
import numpy as np
import librosa

from composition.instrument import Score, Part, Phrase, show
import composition.common as k
from utils import QuantileTransformer

In [ ]:
def fibonacci(n):
    first = 0
    second = 1
    fibs = []
    for _ in range(n):
        first, second = second, first + second
        fibs.append(first)
    
    return fibs

In [ ]:
def fib(n, first=None, second=None):
    if not first:
        first = [1]
    if not second:
        second = [2]
    
    while len(second) < n:
        first, second = second, second + first

    return np.array(second[:n])

In [ ]:
def fill_with_silence(part, n):
    pitches = part.phrases[-1].pitch[-1] * np.ones(n)
    amps = np.ones(n) * -120
    
    part.add_phrase(Phrase(pitches, amps))

In [ ]:
def get_harm(n, stretch=0, base=36):
    f_base = librosa.midi_to_hz(base)
    f_harm = f_base * n ** (1 + stretch)
    return librosa.hz_to_midi(f_harm)

v_get_harm = np.vectorize(get_harm)

In [ ]:
fibonacci(15)

In [ ]:
phi = (np.sqrt(5) + 1) / 2
dur = 7 * 60
b = dur / phi
a = dur - b
a2 = a / phi
a1 = a - a2

b1 = b / phi
b2 = b - b1

print(f'A: {a1:.2f}\" B: {a2:.2f}\" development: {b1:.2f}\" A\'/B\': {b2:.2f}\"')

## Sections

|           | A   | B   | D    | R   | Total |
| --------- | --- | --- | ---- | --- | ----- |
| Rounded   | 61" | 99" | 160" | 99" | 7'    |
| Fibonacci | 55" | 89" | 144" | 89" | 6'17" |

* violin: mean: -43.36, std: 5.23
* viola: mean: -42.76, std: 7.77
* cello: mean: -57.26, std: 6.12
* flute: mean: -55.27, std: 8.37
* flute2: mean: -55.27, std: 8.37
- Violin 55-103
- Viola 48-91
- Cello 36-76
- Flute 60-96
- Flute2 55-91

In [ ]:
dur = 16

In [ ]:
seed_p = np.array((0, 0.7, 0.3, 1.0))
seed_a = np.array((0, 0.375, 0.625, 1.))

In [ ]:
COPPER = 4.236067978

In [ ]:
fig, ax = plt.subplots(1, figsize=(4 * COPPER, 4))
ax.plot(seed_a, seed_p)
ax.set_xticks(seed_a)
plt.show()

# Fractal I

In [ ]:
vln1 = Part('Vln. 1', 'violin')
for i, dr in enumerate([2, 4, 8]):
    seed_d = k.to_delta(seed_a * dr)
    pitches, durs = k.fractal_bpc_floor(seed_p, seed_d, i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 66, 66+12)
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)
    
    if i < 2:
        amp_segments = []
        for d in durs:
            aa = k.triangle(d, 1/d, 1e-7)
            aa = np.pad(aa, (0, k.time_to_step(d) - len(aa)), constant_values=aa[-1])
            amp_segments.append(aa)
        amps = np.concatenate(amp_segments)
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -55, -44)
        amps += k.line_segment(dr, -8, 4)
    else:
        amps = k.line_segment(dr, 0, 1)
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -59, -44)
    
    amps += np.random.randn(*amps.shape) * 0.5
    vln1.add_phrase(Phrase(pitches, amps))
    
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    if i == 2:
        breath_amp = (1 - k.phasor(1+i, 1/(1+i)))
        breath_amp = k.rescale(breath_amp, -58, -46)
        breath_amp[-1] = breath_amp[-2]
    else:
        breath_amp = k.line_segment(1+i, 0, 1) * 9 - 55
    
    breath_amp += np.random.randn(*breath_amp.shape) * 0.5
    vln1.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
vln1.show()
vln1.play()

In [ ]:
vln2 = Part('Vln. 2', 'violin')
for i, dr in enumerate([2, 4, 8]):
    seed_d = k.to_delta(seed_a * dr)
    pitches, durs = k.fractal_bpc_floor(seed_p[::-1], np.roll(seed_d, 1), i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 61, 61+12)
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)
    
    if i < 2:
        amp_segments = []
        for d in durs:
            aa = k.triangle(d, 1/d, 1e-7)
            aa = np.pad(aa, (0, k.time_to_step(d) - len(aa)), constant_values=aa[-1])
            amp_segments.append(aa)
        amps = np.concatenate(amp_segments)
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -55, -44)
        amps += k.line_segment(dr, -12, 4)
    else:
        amps = k.line_segment(dr, 0, 1)
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -59, -44)
    
    amps += np.random.randn(*amps.shape) * 0.5
    vln2.add_phrase(Phrase(pitches, amps))
    
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    if i == 2:
        breath_amp = (1 - k.phasor(1+i, 1/(1+i)))
        breath_amp = k.rescale(breath_amp, -58, -46)
        breath_amp[-1] = breath_amp[-2]
    else:
        breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
    
    breath_amp += np.random.randn(*breath_amp.shape) * 0.5
    vln2.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
vln2.show()
vln2.play()

In [ ]:
volume = -4

vlc = Part('Vlc', 'cello')
for i, dr in enumerate([2, 4, 8]):
    seed_d = k.to_delta(seed_a * dr)
    pitches, durs = k.fractal_bpc_floor(seed_p, seed_d, i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 42, 42+12)
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)
    
    if i < 2:
        amps = np.concatenate([k.triangle(d, 1/d, 0.01) for d in durs])
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -55, -44)
        amps += k.line_segment(dr, -8, 4)
    else:
        amps = k.line_segment(dr, 0, 1)
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -59, -44)
    
    amps += volume
    amps += np.random.randn(*amps.shape) * 0.5
    vlc.add_phrase(Phrase(pitches, amps))
    
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    
    if i == 2:
        breath_amp = (1 - k.phasor(1+i, 1/(1+i))) * 9 - 55
        breath_amp = k.rescale(breath_amp, -59, -46)
        breath_amp[-1] = breath_amp[-2]
    else:
        breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
    
    breath_amp += volume
    breath_amp += np.random.randn(*breath_amp.shape) * 0.5
    vlc.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
vlc.show()
vlc.play()

In [ ]:
volume = 4

vla = Part('Vla', 'viola')
for i, dr in enumerate([2, 4, 8]):
    seed_d = k.to_delta(seed_a * dr)
    pitches, durs = k.fractal_bpc_floor(seed_p[::-1], np.roll(seed_d, 1), i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 49, 49+12)
    # pitches = k.autotune(pitches, 0.95, list(np.random.randint(0, 11, 5)))
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)
    
    if i < 2:
        # amps = k.sinusoid(dr, 2/dr, -43.36, 5.23/2)
        amps = np.concatenate([k.triangle(d, 1/d, 0.01) for d in durs])
        # amps += np.random.normal(0, .5, len(amps))
        # amps = k.to_db_loudness(amps, 'violin', min_loudness=-0.1)
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -55, -44)
        amps += k.line_segment(dr, -8, 4)
        amps = k.rescale(amps, -60, -48)
    else:
        amps = k.line_segment(dr, 0, 1)
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -60, -48)
    
    amps += volume
    amps += np.random.randn(*amps.shape) * 0.5
    vla.add_phrase(Phrase(pitches, amps))
    
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    
    if i == 2:
        breath_amp = (1 - k.phasor(1+i, 1/(1+i))) * 9 - 55
        breath_amp = k.rescale(breath_amp, -66, -47)
        breath_amp[-1] = breath_amp[-2]
    else:
        breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
        breath_amp = k.rescale(breath_amp, -55, -47)

    breath_amp += volume
    breath_amp += np.random.randn(*breath_amp.shape) * 0.5
    vla.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
vla.show()
vla.play()

# Fractal I-II

In [ ]:
flt1 = Part('Flt. 1', 'flute')
silence = Phrase(k.constant(17, 66), k.constant(17, -120))
flt1.add_phrase(silence)
for i, dr in enumerate([1, 2, 4]):
    seed_d = k.to_delta(seed_a * dr)
    pitches, durs = k.fractal_bpc_floor(np.roll(seed_p, 1), np.roll(seed_d, 2), i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 71, 71+12)
    # pitches = k.autotune(pitches, 0.95, list(np.random.randint(0, 11, 5)))
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)

    # amps = k.sinusoid(dr, 2/dr, -43.36, 5.23/2)
    amps = np.concatenate([k.triangle(d, 1/d, 0.01) for d in durs])
    # amps += np.random.normal(0, .5, len(amps))
    # amps = k.to_db_loudness(amps, 'violin', min_loudness=-0.1)
    amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
    amps = k.rescale(amps, -55, -54)
    amps += k.line_segment(dr, 0, 6)
    
    amps += np.random.randn(*amps.shape) * 0.2
    flt1.add_phrase(Phrase(pitches, amps))
    flt1.add_phrase(Phrase(pitches[::-1], amps))
    
    if i == 2:
        i = 3
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
    
    breath_amp += np.random.randn(*breath_amp.shape) * 0.2
    flt1.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
# flt1.show(17)
# flt1.play(17)

In [ ]:
flt2 = Part('Flt. 2', 'flute2')
silence = Phrase(k.constant(17, 66), k.constant(17, -120))
flt2.add_phrase(silence)
for i, dr in enumerate([1, 2, 4]):
    seed_d = k.to_delta(seed_a * dr)
    pitches, durs = k.fractal_bpc_floor(np.roll(seed_p, 2), np.roll(seed_d, 3), i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 66, 66+12)
    # pitches = k.autotune(pitches, 0.95, list(np.random.randint(0, 11, 5)))
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)

    # amps = k.sinusoid(dr, 2/dr, -43.36, 5.23/2)
    amps = np.concatenate([k.triangle(d, 1/d, 0.01) for d in durs])
    # amps += np.random.normal(0, .5, len(amps))
    # amps = k.to_db_loudness(amps, 'violin', min_loudness=-0.1)
    amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
    amps = k.rescale(amps, -58, -57)
    amps += k.line_segment(dr, 0, 6)
    
    amps += np.random.randn(*amps.shape) * 0.2
    flt2.add_phrase(Phrase(pitches, amps))
    flt2.add_phrase(Phrase(pitches[::-1], amps))
    
    if i == 2:
        i = 3
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
    
    breath_amp += np.random.randn(*breath_amp.shape) * 0.2
    flt2.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
# flt2.show(17)
# flt2.play(17)

In [ ]:
# silence, self, retro, chord, self, retro, chord
# skip slience, first self, first retro
_, idx = np.unique(flt1.phrases[4].pitch.round(), return_index=True)
chord = flt1.phrases[4].pitch.round()[np.sort(idx)]
for idx in range(7):
    dur = 2
    pitches = k.constant(dur, chord[(2*idx)%len(chord)])
    attack = k.triangle(dur, 1/dur, ratio=0.01)
#     silence = k.constant(dur/2, 0)
#     amps = np.concatenate([attack, silence])
    amps = k.rescale(attack, -90, -42)
    if idx == 2:
        amps = k.line_segment(dur, -58, -46)
    
    phrase = Phrase(pitches, amps)
    vln1.add_phrase(phrase)

amps = k.line_segment(4, -58, -42)
pitches = k.constant(4, chord[(2*(idx+1))%len(chord)])
phrase = Phrase(pitches, amps)
vln1.add_phrase(phrase)

In [ ]:
# vln1.show(17)
# vln1.play(17)

In [ ]:
# silence, self, retro, chord, self, retro, chord
# skip slience, first self, first retro
_, idx = np.unique(flt1.phrases[4].pitch.round(), return_index=True)
chord = flt1.phrases[4].pitch.round()[np.sort(idx)]
for idx in range(7):
    dur = 2
    pitches = k.constant(dur, chord[(3*idx + 1)%len(chord)])
    attack = k.triangle(dur, 1/dur, ratio=0.01)
#     silence = k.constant(dur/2, 0)
#     amps = np.concatenate([attack, silence])
    amps = k.rescale(attack, -90, -42)
    if idx == 2:
        amps = k.line_segment(dur, -58, -46)
    
    phrase = Phrase(pitches, amps)
    vln2.add_phrase(phrase)

amps = k.line_segment(4, -58, -42)
pitches = k.constant(4, chord[(3*(idx + 1) + 1)%len(chord)])
phrase = Phrase(pitches, amps)
vln2.add_phrase(phrase)

In [ ]:
# vln2.show(17)
# vln2.play(17)

In [ ]:
# silence, self, retro, chord, self, retro, chord
# skip slience, first self, first retro
_, idx = np.unique(flt1.phrases[4].pitch.round(), return_index=True)
chord = flt1.phrases[4].pitch.round()[np.sort(idx)]
for idx in range(7):
    dur = 2
    pitches = k.constant(dur, chord[(5*idx + 2)%len(chord)] - 24)
    pitches[pitches < 48] += 12
    attack = k.triangle(dur, 1/dur, ratio=0.01)
#     silence = k.constant(dur/2, 0)
#     amps = np.concatenate([attack, silence])
    amps = k.rescale(attack, -90, -38)
    if idx == 2:
        amps = k.line_segment(dur, -58, -42)
    
    phrase = Phrase(pitches, amps)
    vla.add_phrase(phrase)

amps = k.line_segment(4, -58, -42)
pitches = k.constant(4, chord[(5*(idx+1) + 2)%len(chord)] - 24)
pitches[pitches < 48] += 12
phrase = Phrase(pitches, amps)
vla.add_phrase(phrase)

In [ ]:
# vla.show(17)
# vla.play(17)

In [ ]:
# silence, self, retro, chord, self, retro, chord
# skip slience, first self, first retro
_, idx = np.unique(flt1.phrases[4].pitch.round(), return_index=True)
chord = flt1.phrases[4].pitch.round()[np.sort(idx)]
for idx in range(7):
    dur = 2
    pitches = k.constant(dur, chord[(3*idx + 3)%len(chord)] - 36)
    pitches[pitches < 36] += 12
    attack = k.triangle(dur, 1/dur, ratio=0.01)
#     silence = k.constant(dur/2, 0)
#     amps = np.concatenate([attack, silence])
    amps = k.rescale(attack, -90, -38)
    if idx == 2:
        amps = k.line_segment(dur, -58, -42)
    
    phrase = Phrase(pitches, amps)
    vlc.add_phrase(phrase)

amps = k.line_segment(4, -58, -42)
pitches = k.constant(4, chord[(3*(idx+1) + 3)%len(chord)] - 36)
pitches[pitches < 36] += 12
phrase = Phrase(pitches, amps)
vlc.add_phrase(phrase)

In [ ]:
# vlc.show(17)
# vlc.play(17)

# Fractal II

In [ ]:
new_seed = np.array([0.0, 0.7, 0.3, 1.0])
new_seed2 = np.array([0.0, 0.8, 0.4, 1.0])

In [ ]:
# -55 -49
# 10 pitches
n_steps = 10*4
pitches = k.fractal(new_seed[::-1], 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durations = durs * 0.06
dur = sum(durations)
silence_dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 66, 66+24)
pitches = k.pad_to_dur(pitches, dur)

amps = k.line_segment(dur, -49, -55)

phrase = Phrase(pitches, amps)
flt1.add_phrase(phrase)

n_steps = 82
pitches = k.fractal(new_seed[::-1], 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 66, 66+24)
pitches = k.pad_to_dur(pitches, dur)

amps = k.constant(dur, -57)

phrase = Phrase(pitches, amps)
flt1.add_phrase(phrase)

n_steps = 28*2
pitches = k.fractal(new_seed2[::-1], 2)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durations = durs * 0.06
dur = sum(durations)
silence_dur2 = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 64, 64+24)
pitches = k.pad_to_dur(pitches, dur)

amps = k.line_segment(dur, -49, -55)

phrase = Phrase(pitches, amps)
flt1.add_phrase(phrase)

n_steps = 82
pitches = k.fractal(new_seed2[::-1], 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durations = durs * 0.06
dur = sum(durations)

last_flt1_pitches = k.rescale(pitches, 64, 64+24)[-10:]

pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 64, 64+24)
pitches = k.pad_to_dur(pitches, dur)

amps = k.constant(dur, -57)

phrase = Phrase(pitches, amps)
flt1.add_phrase(phrase)

In [ ]:
# flt1.show(38)
# flt1.play(38)

In [ ]:
# -51 -58
# 10 pitches
n_steps = 10 * 4
pitches = k.fractal(new_seed, 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durs = np.roll(durs, 1)
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 68, 68+24)

amps = k.line_segment(dur, -51, -58)

pitches = k.pad_to_dur(pitches, dur)

phrase = Phrase(pitches, amps)
flt2.add_phrase(phrase)

n_steps = 82
pitches = k.fractal(new_seed, 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durs = np.roll(durs, 1)
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 68, 68+24)

amps = k.constant(dur, -60)

pitches = k.pad_to_dur(pitches, dur)

phrase = Phrase(pitches, amps)
flt2.add_phrase(phrase)

n_steps = 28 * 2
pitches = k.fractal(new_seed2, 2)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durs = np.roll(durs, 1)
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 70, 70+24)

amps = k.line_segment(dur, -51, -58)

pitches = k.pad_to_dur(pitches, dur)

phrase = Phrase(pitches, amps)
flt2.add_phrase(phrase)

n_steps = 82
pitches = k.fractal(new_seed2, 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durs = np.roll(durs, 1)
durations = durs * 0.06
dur = sum(durations)

last_flt2_pitches = k.rescale(pitches, 70, 70+24)[-10:]

pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 70, 70+24)

amps = k.constant(dur, -60)

pitches = k.pad_to_dur(pitches, dur)

phrase = Phrase(pitches, amps)
flt2.add_phrase(phrase)

In [ ]:
# flt2.show(38)
# flt2.play(38)

In [ ]:
silence_steps = int(silence_dur*k.SECOND)
# part.add_phrase(Phrase(np.zeros(silence_steps)+55, np.zeros(silence_steps)-100))
pitches = k.constant(silence_dur, np.round(vln1.phrases[-1].pitch[-1]))
amps = k.triangle(silence_dur, 4/silence_dur, 0.05, -90, -44)
vln1.add_phrase(Phrase(pitches, amps))

# 82 pitches
pitches = k.fractal(new_seed, 3)
durs = fib(len(pitches))
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 55, 55+24)
pitches = k.pad_to_dur(pitches, dur)

amps = k.constant(dur, -44)

phrase = Phrase(pitches, amps)
vln1.add_phrase(phrase)

silence_steps = int(silence_dur2*k.SECOND)
# part.add_phrase(Phrase(np.zeros(silence_steps)+55, np.zeros(silence_steps)-100))
pitches = k.constant(silence_dur2, np.round(vln1.phrases[-1].pitch[-1]))
amps = k.triangle(silence_dur2, 2/silence_dur2, 0.05, -90, -44)
vln1.add_phrase(Phrase(pitches, amps))

# 82 pitches
new_seed = np.array([0.0, 0.7, 0.3, 1.0])
pitches = k.fractal(new_seed2, 3)
durs = fib(len(pitches))
durations = durs * 0.06
dur = sum(durations)

last_vln1_pitches = k.rescale(pitches, 57, 57+24)[-10:]

pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 57, 57+24)
pitches = k.pad_to_dur(pitches, dur)

amps = k.constant(dur, -44)

phrase = Phrase(pitches, amps)
vln1.add_phrase(phrase)

In [ ]:
# vln1.show(38)
# vln1.play(38)

In [ ]:
silence_steps = int(silence_dur*k.SECOND)
# part2.add_phrase(Phrase(np.zeros(silence_steps)+55, np.zeros(silence_steps)-100))
pitches = k.constant(silence_dur, np.round(vln2.phrases[-1].pitch[-1]))
amps = k.triangle(silence_dur, 4/silence_dur, 0.2, -90, -44)
vln2.add_phrase(Phrase(pitches, amps))

# 82 pitches
pitches = k.fractal(new_seed[::-1], 3)
durs = fib(len(pitches))
durs = np.roll(durs, 1)
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 60, 60+24)

amps = k.constant(dur, -44)

pitches = k.pad_to_dur(pitches, dur)

phrase = Phrase(pitches, amps)
vln2.add_phrase(phrase)

silence_steps = int(silence_dur2*k.SECOND)
# part2.add_phrase(Phrase(np.zeros(silence_steps)+55, np.zeros(silence_steps)-100))
pitches = k.constant(silence_dur2, np.round(vln2.phrases[-1].pitch[-1]))
amps = k.triangle(silence_dur2, 2/silence_dur2, 0.2, -90, -44)
vln2.add_phrase(Phrase(pitches, amps))

# 82 pitches
pitches = k.fractal(new_seed2[::-1], 3)
durs = fib(len(pitches))
durs = np.roll(durs, 1)
durations = durs * 0.06
dur = sum(durations)

last_vln2_pitches = k.rescale(pitches, 62, 62+24)[-10:]

pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 62, 62+24)

amps = k.constant(dur, -44)

pitches = k.pad_to_dur(pitches, dur)

phrase = Phrase(pitches, amps)
vln2.add_phrase(phrase)

In [ ]:
# vln2.show(38)
# vln2.play(38)

In [ ]:
fill_with_silence(vla, len(vln1) - len(vla))
fill_with_silence(vlc, len(vln1) - len(vlc))

In [ ]:
for p in (last_flt1_pitches, last_flt2_pitches, last_vln1_pitches, last_vln2_pitches):
    print(p)

# Transition

In [ ]:
n_repeat = 32
pitches = n_repeat * [last_flt1_pitches[1], last_flt1_pitches[3]]
durs = fib(n_repeat) * 0.06
pitches = k.figure(pitches, durations)

amps = n_repeat * [-48, -50]
amps = k.figure(amps, durations)
amps += np.linspace(0, -24, len(amps))

phrase = Phrase(pitches, amps)
flt1.add_phrase(phrase)

In [ ]:
# flt1.show(63.312)
# flt1.play(63.312)

In [ ]:
n_repeat = 32
pitches = n_repeat * [last_flt2_pitches[2], last_flt2_pitches[4]]
durs = fib(n_repeat) * 0.06
pitches = k.figure(pitches, durations)

amps = n_repeat * [-48, -50]
amps = k.figure(amps, durations)
amps += np.linspace(0, -24, len(amps))

phrase = Phrase(pitches, amps)
flt2.add_phrase(phrase)

In [ ]:
# flt2.show(63.312)
# flt2.play(63.312)

In [ ]:
n_repeat = 32
pitches = n_repeat * [last_vln1_pitches[3], last_vln1_pitches[5]]
durs = fib(n_repeat) * 0.06
pitches = k.figure(pitches, durations)

amps = n_repeat * [-48, -50]
amps = k.figure(amps, durations)
amps += np.linspace(0, -24, len(amps))

phrase = Phrase(pitches, amps)
vln1.add_phrase(phrase)

In [ ]:
# vln1.show(63.312)
# vln1.play(63.312)

In [ ]:
n_repeat = 32
pitches = n_repeat * [last_vln2_pitches[4], last_vln2_pitches[6]]
durs = fib(n_repeat) * 0.06
pitches = k.figure(pitches, durations)

amps = n_repeat * [-48, -50]
amps = k.figure(amps, durations)
amps += np.linspace(0, -24, len(amps))

phrase = Phrase(pitches, amps)
vln2.add_phrase(phrase)

In [ ]:
# vln2.show(63.312)
# vln2.play(63.312)

# Harmonics I

In [ ]:
higher_harms = k.harmonics_to_pitches(36, [2, 11, 13, 15, 17, 19, 21, 23, 25, 27])[1:]
stretched_higher_harms = k.harmonics_to_pitches(36, [n ** 1.02 for n in [2, 11, 13, 15, 17, 19, 21, 23, 25, 27]])[1:]

In [ ]:
p1, p2 = higher_harms[0], stretched_higher_harms[0]

pitch = np.concatenate([
    k.constant(4, p1) + k.sinusoid(4, 2, amp=k.line_segment(4, 0.3, 0), phase=np.random.uniform(0, 1)),
    k.line_segment(6, p1, p2),
    k.constant(2, p2) + np.concatenate([
        k.sinusoid(1, 2, amp=k.line_segment(1, 0, 0.3), phase=np.random.uniform(0, 1)),
        k.sinusoid(1, 2, amp=k.line_segment(1, 0.3, 0), phase=np.random.uniform(0, 1))
    ]),
    k.line_segment(4, p2, p1),
    k.constant(6, p1) + np.concatenate([
        k.sinusoid(3, 2, amp=k.line_segment(3, 0, 0.3), phase=np.random.uniform(0, 1)),
        k.sinusoid(3, 2, amp=k.line_segment(3, 0.3, 0), phase=np.random.uniform(0, 1))
    ]),
    k.line_segment(6, p1, p2),
    k.constant(4, p2) + np.concatenate([
        k.sinusoid(2, 2, amp=k.line_segment(2, 0, 0.3), phase=np.random.uniform(0, 1)),
        k.sinusoid(2, 2, amp=k.line_segment(2, 0.3, 0), phase=np.random.uniform(0, 1))
    ]),
    k.line_segment(8, p2, p1),
    k.constant(4, p1) + k.sinusoid(4, 2, amp=k.line_segment(4, 0, 0.3), phase=np.random.uniform(0, 1)),
])

loudness = np.concatenate([
    k.constant(4, 0.01),
    k.line_segment(12, 0.01, 0.2),
    k.line_segment(12, 0.2, 0.05),
    k.line_segment(12, 0.05, 0.9),
    k.constant(4, 0.9)
])
loudness *= 10.
loudness += k.sweeping_triangle(44, 0.05, 4.25, 0.1, highest=8, phase=0.5)
loudness += np.random.randn(*loudness.shape) * 0.05

loudness[:k.SECOND//10] = k.line_segment(1/10, 0, loudness[-k.SECOND//10])
loudness[-k.SECOND//10:] = k.line_segment(1/10, loudness[k.SECOND//10], 0)

loudness = k.to_db_loudness(loudness, 'viola', -2, 0.)
loudness = np.clip(loudness, -np.inf, -30)
# loudness -= 6

phrase = Phrase(pitch, loudness)
vla.add_phrase(phrase)

In [ ]:
vla.phrases[-1].show()
vla.phrases[-1].play('viola')

In [ ]:
# vla.show(63.312)
# vla.play(63.312)

In [ ]:
pitch = k.constant(44, 36)
pitch += k.sinusoid(44, 4, 0, 0.125)

loudness = k.line_segment(44, 0.01, 200)
loudness += k.sweeping_triangle(44, 0.2, 4, ratio=0.1, lowest=0, highest=75)
loudness[-8*k.SECOND:] = k.line_segment(8, 1, 0)**2 * 50
loudness[-k.SECOND//10:] = k.line_segment(1/10, loudness[-k.SECOND//10], 0)
loudness[:k.SECOND//10] = k.line_segment(1/10, 0, loudness[k.SECOND//10])

loudness = k.to_db_loudness(loudness, 'cello', 7)
loudness = np.clip(loudness, -np.inf, -30)
# loudness -= 6

phrase = Phrase(pitch, loudness)
vlc.add_phrase(phrase)

In [ ]:
vlc.phrases[-1].show()
vlc.phrases[-1].play('cello')

In [ ]:
# vlc.show(63.312)
# vlc.play(63.312)

In [ ]:
p1, p2 = higher_harms[1], stretched_higher_harms[1]

pitch = np.concatenate([
    k.constant(4, p1) + k.sinusoid(4, 2, amp=k.line_segment(4, 0.3, 0), phase=np.random.uniform(0, 1)),
    k.line_segment(6, p1, p2),
    k.constant(2, p2) + np.concatenate([
        k.sinusoid(1, 2, amp=k.line_segment(1, 0, 0.3), phase=np.random.uniform(0, 1)),
        k.sinusoid(1, 2, amp=k.line_segment(1, 0.3, 0), phase=np.random.uniform(0, 1))
    ]),
    k.line_segment(4, p2, p1),
    k.constant(6, p1) + np.concatenate([
        k.sinusoid(3, 2, amp=k.line_segment(3, 0, 0.3), phase=np.random.uniform(0, 1)),
        k.sinusoid(3, 2, amp=k.line_segment(3, 0.3, 0), phase=np.random.uniform(0, 1))
    ]),
    k.line_segment(6, p1, p2),
    k.constant(4, p2) + np.concatenate([
        k.sinusoid(2, 2, amp=k.line_segment(2, 0, 0.3), phase=np.random.uniform(0, 1)),
        k.sinusoid(2, 2, amp=k.line_segment(2, 0.3, 0), phase=np.random.uniform(0, 1))
    ]),
    k.line_segment(8, p2, p1),
    k.constant(4, p1) + k.sinusoid(4, 2, amp=k.line_segment(4, 0, 0.3), phase=np.random.uniform(0, 1)),
])

pitch[:4*k.SECOND] += k.sinusoid(4, 2, amp=k.line_segment(4, 0.3, 0), phase=np.random.uniform(0, 1))
pitch[-4*k.SECOND:] += k.sinusoid(4, 2, amp=k.line_segment(4, 0, 0.3), phase=np.random.uniform(0, 1))

loudness = np.concatenate([
    k.constant(4, 0.0),
    k.line_segment(12, 0.0, 0.2),
    k.line_segment(12, 0.2, 0.05),
    k.line_segment(12, 0.05, 0.9),
    k.line_segment(2, 0.9, 0.),
    k.constant(2, 0.)
])
loudness *= 10.
loudness += k.sweeping_triangle(44, 0.1, 4, 0.1, highest=8, lowest=-10)
loudness += np.random.randn(*loudness.shape) * 0.3
loudness += k.line_segment(44, 10, 0)

loudness[:k.SECOND//10] = k.line_segment(1/10, loudness.min(), loudness[-k.SECOND//10])
loudness[-k.SECOND//10:] = k.line_segment(1/10, loudness[k.SECOND//10], loudness.min())

loudness = loudness.clip(min=2.)
loudness = k.to_db_loudness(loudness, 'violin', 8, 0.)

phrase = Phrase(pitch, loudness)
vln1.add_phrase(phrase)

In [ ]:
vln1.show(69.552)
vln1.play(69.552)

In [ ]:
p1, p2 = higher_harms[2], stretched_higher_harms[2]

pitch = np.concatenate([
    k.constant(4, p1) + k.sinusoid(4, 2, amp=k.line_segment(4, 0.3, 0), phase=np.random.uniform(0, 1)),
    k.line_segment(6, p1, p2),
    k.constant(2, p2) + np.concatenate([
        k.sinusoid(1, 2, amp=k.line_segment(1, 0, 0.3), phase=np.random.uniform(0, 1)),
        k.sinusoid(1, 2, amp=k.line_segment(1, 0.3, 0), phase=np.random.uniform(0, 1))
    ]),
    k.line_segment(4, p2, p1),
    k.constant(6, p1) + np.concatenate([
        k.sinusoid(3, 2, amp=k.line_segment(3, 0, 0.3), phase=np.random.uniform(0, 1)),
        k.sinusoid(3, 2, amp=k.line_segment(3, 0.3, 0), phase=np.random.uniform(0, 1))
    ]),
    k.line_segment(6, p1, p2),
    k.constant(4, p2) + np.concatenate([
        k.sinusoid(2, 2, amp=k.line_segment(2, 0, 0.3), phase=np.random.uniform(0, 1)),
        k.sinusoid(2, 2, amp=k.line_segment(2, 0.3, 0), phase=np.random.uniform(0, 1))
    ]),
    k.line_segment(8, p2, p1),
    k.constant(4, p1) + k.sinusoid(4, 2, amp=k.line_segment(4, 0, 0.3), phase=np.random.uniform(0, 1)),
])

loudness = np.concatenate([
    k.constant(4, 0.01),
    k.line_segment(12, 0.01, 0.2),
    k.line_segment(12, 0.2, 0.05),
    k.line_segment(12, 0.05, 0.9),
    k.constant(4, 0.9)
])
loudness *= 10.
loudness += k.sweeping_triangle(44, 0.15, 3.75, 0.1, highest=8, phase=0.25)
loudness += np.random.randn(*loudness.shape) * 0.3

loudness[:k.SECOND//10] = k.line_segment(1/10, 0, loudness[-k.SECOND//10])
loudness[-k.SECOND//10:] = k.line_segment(1/10, loudness[k.SECOND//10], 0)

loudness = k.to_db_loudness(loudness, 'violin', -6, 0.)
loudness = np.clip(loudness, -np.inf, -30)
loudness -= 4

phrase = Phrase(pitch, loudness)
vln2.add_phrase(phrase)

In [ ]:
vln2.phrases[-1].show()
vln2.phrases[-1].play('viola')

In [ ]:
# vln2.show(69.552)
# vln2.play(69.552)

In [ ]:
silence = 7

In [ ]:
pitch = k.constant(silence, flt1.phrases[-1].pitch[-1])
loudness = k.constant(silence, -120)

phrase = Phrase(pitch, loudness)
flt1.add_phrase(phrase)

ps = np.array(higher_harms[-5:])

amp = (ps[-1] - ps[0]) / 2
center = ps[0] + amp

pitch = k.sinusoid(44, (1/2)*phi, center, amp / 3)
pitch += k.sinusoid(44, (1/3)*phi, 0, amp / 3)
pitch += k.sinusoid(44, (1/5)*phi, 0, amp / 3)
pitch = k.autotune_explicit(pitch, 0.9, ps)

loudness = k.bpc([0, 2, 1, 1.5, 0.], [12, 24, 4, 4])

loudness = k.to_db_loudness(loudness, 'flute')

phrase = Phrase(pitch, loudness)
flt1.add_phrase(phrase)

In [ ]:
# 76.552
# flt1.phrases[-1].show()
# flt1.phrases[-1].play('flute')

In [ ]:
pitch = k.constant(silence, flt2.phrases[-1].pitch[-1])
loudness = k.constant(silence, -120)

phrase = Phrase(pitch, loudness)
flt2.add_phrase(phrase)

ps = np.array(higher_harms[-9:-4])

amp = (ps[-1] - ps[0]) / 2
center = ps[0] + amp

pitch = k.sinusoid(44, 1/2, center, amp / 3)
pitch += k.sinusoid(44, 1/3, 0, amp / 3)
pitch += k.sinusoid(44, 1/5, 0, amp / 3)
pitch = k.autotune_explicit(pitch, 0.9, ps)

loudness = k.bpc([0, 1, 0.5, 0.75, 0.], [12, 24, 4, 4])
loudness += k.sweeping_triangle(44, 0.1, 0.5, 0.5, lowest = 0.2)
loudness[-k.SECOND//10:] = k.line_segment(1/10, loudness[-k.SECOND//10], 0)
loudness[:k.SECOND//10] = k.line_segment(1/10, 0, loudness[1*k.SECOND//10])

loudness = k.to_db_loudness(loudness, 'flute2', 4)

phrase = Phrase(pitch, loudness)
flt2.add_phrase(phrase)

In [ ]:
# flt2.phrases[-1].show()
# flt2.phrases[-1].play('flute2')

In [ ]:
for p in (vln1, vln2, vla, vlc, flt1, flt2):
    fill_with_silence(p, k.SECOND*128 - len(p))

# Harmonics II

In [ ]:
omegas = [(phi ** n) / 4 for n in range(6)]
adsr = np.array([0.1, 0.3, 0.4, 0.2])
cello_dur = 8/phi
first_stretch = k.time_to_step(26)
second_stretch = k.time_to_step(38)
third_stretch = k.time_to_step(44)
emphasis = (-8, 8)
breathe = (-8, 0)

for idx, (w, p) in enumerate(zip(omegas, [vlc, vla, flt1, flt2, vln1, vln2])):
    # base pitches
    n_harm = 2 * idx + 1
    pitch = k.constant(60, get_harm(n_harm))
    
    # base amplitudes
    # TODO: there are audio artifacts if both attack and decay are close to zero
    amp = np.concatenate([k.rescale(k.random_adsr(1/w, 0.6), -76, -34) for _ in range(int(60*w) + 1)])
    amp = k.pad_to_dur(amp, 60)
    amp = amp[:len(pitch)]
    amp += k.triangle(60, 1/4, 0.5, *breathe, idx/6)
    
    # part specific modifications
    if p.part_name == 'Vla':
        amp += 4
        amp[:int(cello_dur * 2 * k.SECOND)] = -120
        amp[k.time_to_step(57):] = -120
    elif p.part_name == 'Vln. 1':
        amp[:int(cello_dur * 3 * k.SECOND)] = -120
        amp[k.time_to_step(57):] = -120
    elif p.part_name == 'Vln. 2':
        amp[:int(cello_dur * 4 * k.SECOND)] = -120
        amp[k.time_to_step(57):] = -120
    elif p.part_name == 'Flt. 1':
        amp += 4
        amp[k.time_to_step(57):] = -120
    elif p.part_name == 'Flt. 2':
        amp[k.time_to_step(57):] = -120
    
    # harmonic stretching after 28 seconds
    amount = 0.05
    stretch = k.line_segment(3, 0, amount)
    relax = k.line_segment(3, amount, 0)
    p_stretch = v_get_harm(n_harm, stretch)
    p_relax = v_get_harm(n_harm, relax)
    
    end = first_stretch + len(p_stretch)
    end2 = end + len(p_relax)
    pitch[first_stretch:end] = p_stretch
    pitch[end:end2] = p_relax
    
    amp[first_stretch:end] += k.line_segment(3, *emphasis)
    amp[end:end2] += k.line_segment(3, *emphasis[::-1])
    
    # second stretch at 38
    amount = 0.025
    stretch = k.line_segment(1, 0, amount)
    relax = k.line_segment(2, amount, 0)
    p_stretch = v_get_harm(n_harm, stretch)
    p_relax = v_get_harm(n_harm, relax)
    
    end = second_stretch + len(p_stretch)
    end2 = end + len(p_relax)
    pitch[second_stretch:end] = p_stretch
    pitch[end:end2] = p_relax
    
    amp[second_stretch:end] += k.line_segment(1, *emphasis)
    amp[end:end2] += k.line_segment(2, *emphasis[::-1])
    
    # third stretch at 46
    amount = -0.05
    stretch = k.line_segment(8, 0, amount)
    relax = k.line_segment(4, amount, 0)
    p_stretch = v_get_harm(n_harm, stretch)
    p_relax = v_get_harm(n_harm, relax)
    
    end = third_stretch + len(p_stretch)
    end2 = end + len(p_relax)
    pitch[third_stretch:end] = p_stretch
    pitch[end:end2] = p_relax
    
    amp[third_stretch:end] += k.line_segment(8, *emphasis[::-1])
    amp[end:end2] += k.line_segment(4, *emphasis)
    
    # build phrase
    phrase = Phrase(pitch, amp)
    p.add_phrase(phrase)

In [ ]:
vlc.phrases[-1].show()

In [ ]:
score = Score([vln1, vln2, vla, vlc, flt1, flt2])

In [ ]:
score.save('fractus-duo', './audio-data/december/')

In [ ]:
for p in [vln1, vln2, vla, vlc, flt1, flt2]:
    del p.phrases[-1]

In [ ]:
# parts = (vln1, vln2)
# figure, axes = plt.subplots(2)
# print(type(axes))
# for p in parts:
#     fig = plt.figure(p.show())
#     ax = fig.axes
#     print(type(fig.axes))
#     gs = ax[0, 1].get_gridspec()
#     figure.add_subplot(gs)